In [46]:
import requests
import pandas as pd

# 1. API endpoint
url = "https://api.census.gov/data/2023/pep/charv"

# 2. Parameters
params = {
    "get": "NAME,STATE,YEAR,POP,AGE,SEX",
    "for": "state:*"
}

# 3. Send request
response = requests.get(url, params=params)

# 4. Parse JSON
data = response.json()

# 5. Convert to DataFrame
pop_df = pd.DataFrame(data[1:], columns=data[0])

pop_df.sample(5)


,NAME,STATE,YEAR,POP,AGE,SEX,state
17238,District of Columbia,11,2022,70537,5999,2,11
18320,Florida,12,2020,119299,0600,1,12
38189,Maine,23,2023,9960,7000,2,23
21698,Hawaii,15,2022,16861,5400,0,15
78854,Virginia,51,2021,1714098,1544,2,51


In [47]:
pop_df.to_csv(
    "population_dataset.csv",
    index=False
)

In [48]:
pop_df.sample(5)

,NAME,STATE,YEAR,POP,AGE,SEX,state
50380,Nebraska,31,2020,13803,3800,1,31
73655,South Dakota,46,2020,11372,5600,0,46
3010,Alaska,02,2020,4783,1500,2,02
63952,Oklahoma,40,2022,138908,0509,1,40
53772,New Jersey,34,2021,55500,2300,1,34


In [49]:
print(pop_df.shape)
print(type(pop_df.AGE[0]))

(90480, 7)
<class 'str'>


In [50]:
def fetch_acs_median_income_state(year):
    """
    grab ACS 1-year meadian household income in past 12 months for all states.
    endpoint: https://api.census.gov/data/{year}/acs/acs1
    variable: B19013_001E
    """
    url = f"https://api.census.gov/data/{year}/acs/acs1"
    params = {
        "get": "NAME,B19013_001E",
        "for": "state:*",
    }

    r = requests.get(url, params=params, timeout=60)

    # raise error if not 200
    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:300]}")

    data = r.json()
    df = pd.DataFrame(data[1:], columns=data[0])

    # clean
    df["YEAR"] = year
    df = df.rename(columns={"state": "STATE", "B19013_001E": "median_hh_income"})
    df["STATE"] = df["STATE"].astype(str).str.zfill(2)
    df["median_hh_income"] = pd.to_numeric(df["median_hh_income"], errors="coerce")

    return df[["STATE", "YEAR", "NAME", "median_hh_income"]]


years = [2021, 2022, 2023]
acs_income = pd.concat([fetch_acs_median_income_state(y) for y in years], ignore_index=True)

acs_income.head()


,STATE,YEAR,NAME,median_hh_income
0,01,2021,Alabama,53913
1,72,2021,Puerto Rico,22237
2,04,2021,Arizona,69056
3,05,2021,Arkansas,52528
4,06,2021,California,84907


In [51]:
acs_income.to_csv(
    "median_income_state.csv",
    index=False
)

In [52]:
print(len(acs_income.STATE.unique()))
print(acs_income.STATE.unique())


52
['01' '72' '04' '05' '06' '08' '09' '10' '11' '12' '13' '15' '16' '17'
 '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30' '31'
 '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '44' '45' '46'
 '47' '48' '49' '50' '51' '53' '54' '55' '56' '02']


In [53]:
pop_df["STATE"] = pop_df["STATE"].astype(str).str.zfill(2)
acs_income["STATE"] = acs_income["STATE"].astype(str).str.zfill(2)

pop_df["YEAR"] = pop_df["YEAR"].astype(int)
acs_income["YEAR"] = acs_income["YEAR"].astype(int)

merged_df = pop_df.merge(
    acs_income[["STATE", "YEAR", "median_hh_income"]],
    on=["STATE", "YEAR"],
    how="left"
)

merged_df.to_csv(
    "merged_dataset.csv",
    index=False
)

In [54]:
merged_df.sample(5)

,NAME,STATE,YEAR,POP,AGE,SEX,state,median_hh_income
82101,Utah,49,2020,13875,6500,1,49,NaN
78145,Texas,48,2020,1070690,1519,1,48,NaN
84214,Puerto Rico,72,2023,129936,0513,1,72,25621.0
50203,Nevada,32,2020,81127,1417,1,32,NaN
9096,California,06,2022,243054,4700,1,06,91551.0
